In [ ]:
from wtforms import Form, StringField, SelectField
from flask_wtf import FlaskForm
from flask import Flask, redirect, url_for, request,flash,render_template

import csv
import pandas as pd
import os
from IPython.display import HTML
import matplotlib.pyplot as plt
#driver_csv = pd.read_csv(r"E:\ESM\BlockedWeek\f1db_csv\drivers.csv")
#driver_ranking = pd.read_csv(r"E:\ESM\BlockedWeek\f1db_csv\driver_standings.csv")

driver_csv = pd.read_csv(r".\f1db_csv\drivers.csv")
driver_ranking = pd.read_csv(r".\f1db_csv\driver_standings.csv")

# print(driver_csv.head())
# print(driver_ranking.head())

class DriverForm(FlaskForm):
    driverRef = StringField('driverRef')
    number = StringField('number')
    code = StringField('code')
    forename = StringField('forename')
    surname = StringField('surname')
    dob = StringField('dob')
    nationality = StringField('nationality')
    url = StringField('url')
    
class DriverID(FlaskForm):
    driverId = StringField('driverID')

driver_csv = pd.read_csv(r".\f1db_csv\drivers.csv")
driver_ranking = pd.read_csv(r".\f1db_csv\driver_standings.csv")
driverDetails = pd.read_csv(r".\f1db_csv\drivers.csv")

def enterDriverDetails(driverDetails, tempList):
    k = 0
    print("Inside the main saving function..")
       #for k in range(1, len(driverDetails.columns)):
    #    print(tempList[k])
    maxId = driverDetails['driverId'].max() + 1
    data = []
    columns = []
    for i in range(len(tempList) + 1):
        if i == 0:
            data.append(maxId)
        else:
            data.append(tempList[i - 1])
        columns.append(driverDetails.columns[i])
    newFrame = [{columns[0]: data[0], columns[1]: data[1], columns[2]: data[2], columns[3]: data[3], columns[4]: data[4], columns[5]: data[5], columns[6]: data[6], columns[7]: data[7], columns[8]: data[8]}]
    newFrame = pd.DataFrame(newFrame)
    driverDetails = driverDetails.append(newFrame, ignore_index = True)
    #driverDetails = driverDetails.drop(columns=driverDetails.columns[0])
    html_render = driverDetails.to_html()
    csv_file = driverDetails.to_csv(r".\f1db_csv\drivers.csv", index=False)
    
def homeView():
    print("Inside the home page")
    
    return render_template('home.html')



def get_driver_info(driverId):
    merged_dataframe = pd.merge(driver_csv,driver_ranking,on ='driverId', how = 'left')
    filtered_data=merged_dataframe.loc[merged_dataframe['driverId'] == driverId]
    #render_template('driver.html',content="test")
    return filtered_data
    

def plot_driver_info():
    driver_csv['nationality'].value_counts().plot(kind='bar');
    plt.figure(1)
    plt.xlabel("Country")
    plt.ylabel("Driver count")
    #plt.show()
    plt.savefig(r".\static\2.png")
    plt.close()
    

def plot_driver_position_chart(driverId):
    #import pdb
    #pdb.set_trace()
    driver_details =  get_driver_info(driverId)
    driver_details['position'].value_counts().plot(kind='bar');
    #plt.figure(2)
    #plt.xlabel("Position")
   # plt.ylabel("Count")
    #plt.show()
    plt.savefig(r".\static\1.png")

    
def enterDriver():
    driver_form = DriverForm()
    print("Inside the enterDriver details function...")
    if driver_form.validate_on_submit():
        driverDetails = pd.read_csv(r".\f1db_csv\drivers.csv")
        tempList = [driver_form.driverRef.data, driver_form.number.data, driver_form.code.data, driver_form.forename.data, driver_form.surname.data, driver_form.dob.data, driver_form.nationality.data, driver_form.url.data]
        enterDriverDetails(driverDetails, tempList)
        print('pandas frame created .. !')
        return redirect(url_for('homePage'))
    return render_template('updateDriver.html',driver_form=driver_form)
  
def remove(driverDetails, getDriverID):
    #getDriverID = input('Enter the Driver ID: ')
    driverIDFound = False
    for index, row in driverDetails.iterrows():
        if driverDetails.loc[index,'driverId'] == int(getDriverID):
            reqRow = index
            driverIDFound = True
    driverDetails = driverDetails.drop([reqRow])
    html_render = driverDetails.to_html()
    csv_file = driverDetails.to_csv(r".\f1db_csv\drivers.csv", index=False)
    
def deleteDriver():
    id_form = DriverID()
    print("Inside the Delete Function")
    if id_form.validate_on_submit():
        driverDetails = pd.read_csv(r".\f1db_csv\drivers.csv")
        driverId = id_form.driverId.data
        remove(driverDetails, driverId)
        print("Entry removed..")
        return redirect(url_for('homePage'))
    return render_template('delete.html',id_form=id_form)

def displayView(driverDetails):
    print("Inside the display page.. ")
    html_render = driverDetails.to_html()
    html_file = open(r'templates\display.html', "w")
    html_file.write(html_render)
    back_button = '<form method ="post" action="http://localhost:81/"><button type="submit" name="Home Page" value="Home Page"> Home Page </button></form>'
    html_file.write(back_button)
    html_file.close()
    return html_render


app = Flask(__name__)
app.config['SECRET_KEY'] = 'C2HWGVoMGfNTBsrYQg8EcMrdTimkZfAb'

@app.route('/', methods=['GET', 'POST'])
def homePage():
    return homeView()

@app.route('/result',methods = ['POST', 'GET'])
def result():
   if request.method == 'POST':
      result = request.form
      a = int(request.form['DriverId'])
      filtered_data=get_driver_info(a)
      filtered_data.to_html(r".\templates\driver.html")
   return render_template('driver.html',result = result)
    
    
@app.route('/driver_stats',methods = ['POST', 'GET'])
def driver_stats():
   if request.method == 'POST':
      result = request.form
      a = int(request.form['DriverId'])
      if request.form.get("Show Driver Statistics"):
        plot_driver_position_chart(a)
        print("a=",a)
        return render_template('driver_stats.html')

@app.route('/new_driver', methods=['GET', 'POST'])
def new_driver():
    """
    Add a new entry
    """
    return enterDriver()

@app.route('/delete_driver', methods=['GET','POST','DELETE'])
def delete_driver():
    """
    Add a new entry
    """
    return deleteDriver()
    

@app.route('/display', methods=['GET', 'POST'])
def displayPage():
    driverDetails = pd.read_csv(r".\f1db_csv\drivers.csv")
    return displayView(driverDetails)

@app.route('/hello',methods = ['POST', 'GET'])
def hello():
    if request.method == "POST":
        if request.form.get("Show country status"):
            plot_driver_info()
            return render_template('driver_country_image.html')
    
    


app.run(host='0.0.0.0', port=81)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:81/ (Press CTRL+C to quit)
127.0.0.1 - - [22/May/2021 13:17:45] "GET / HTTP/1.1" 200 -


Inside the home page
Inside the display page.. 


127.0.0.1 - - [22/May/2021 13:17:48] "POST /display HTTP/1.1" 200 -
